- sections : '정치', '경제', '사회'
- sections = ["pol", "eco","soc"]
- section_ids = ["100", "101","102"]

In [45]:
import requests
from bs4 import BeautifulSoup
import bs4.element
import datetime
import warnings
warnings.filterwarnings('ignore')
import re
import pandas as pd
import time
from datetime import datetime

In [46]:
# ConnectionError방지
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}

# BeautifulSoup 객체 생성
def get_soup_obj(url):
    res = requests.get(url, headers = headers, verify=False)
    soup = BeautifulSoup(res.text,'lxml')
    
    return soup

In [47]:
# 뉴스의 기본 정보 가져오기
def get_top3_news_info():
    news_urls =[]
    
    # 해당 분야 상위 뉴스 목록 주소
    sec_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=101"
    
    # 해당 분야 상위 뉴스 HTML 가져오기
    soup = get_soup_obj(sec_url)
  
    # 해당 분야 상위 뉴스 3개 가져오기

    lis3 = soup.find('ul', class_='type06_headline').find_all("li", limit=3)
    for li in lis3:
        news_url = li.a.attrs.get('href')
        news_urls.append(news_url)
    
    print(news_urls)
    return news_urls

In [48]:
# 상위 3개 뉴스 크롤링
news_urls = get_top3_news_info()

news_titles = []
news_contents =[]
news_dates = []

for url in news_urls:
    news_html = get_soup_obj(url)

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")

    # 뉴스 본문 가져오기
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
        
    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    html_date = news_html.select_one("div#ct > div.media_end_head > div.media_end_head_info > div span")
    news_date = html_date.attrs['data-date-time']
    news_dates.append(news_date)

['https://n.news.naver.com/mnews/article/009/0005154459?sid=101', 'https://n.news.naver.com/mnews/article/422/0000607237?sid=101', 'https://n.news.naver.com/mnews/article/214/0001283877?sid=101']


In [49]:
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':news_urls,'content':news_contents})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

중복 제거 후 행 개수:  3


In [50]:
def add_df() :
    a = pd.DataFrame({'date':news_dates,'title':news_titles,'link':news_urls,'content':news_contents})

    #중복 행 지우기
    a = a.drop_duplicates(keep='first',ignore_index=True)
    print("중복 제거 후 행 개수: ",len(a))

    pd.concat([news_df, a])

    # #중복 행 지우기
    # news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
    # print("중복 제거 후 행 개수: ",len(news_df))

    print(news_df.head())

In [51]:
t=1
while t<5 :
    news_urls = get_top3_news_info()

    news_titles = []
    news_contents =[]
    news_dates = []

    # titles = []
    # contents =[]
    # dates = []

    for url in news_urls:
        news_html = get_soup_obj(url)

        # 뉴스 제목 가져오기
        title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        if title == None:
            title = news_html.select_one("#content > div.end_ct > div > h2")

        # 뉴스 본문 가져오기
        content = news_html.select("div#dic_area")
        if content == []:
            content = news_html.select("#articeBody")
            
        # 기사 텍스트만 가져오기
        # list합치기
        content = ''.join(str(content))

        # html태그제거 및 텍스트 다듬기
        pattern1 = '<[^>]*>'
        title = re.sub(pattern=pattern1, repl='', string=str(title))
        content = re.sub(pattern=pattern1, repl='', string=content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')

        news_titles.append(title)
        news_contents.append(content)

        html_date = news_html.select_one("div#ct > div.media_end_head > div.media_end_head_info > div span")
        news_date = html_date.attrs['data-date-time']
        news_dates.append(news_date)
    
    add_df()

    time.sleep(30)

    t += 1

['https://n.news.naver.com/mnews/article/009/0005154459?sid=101', 'https://n.news.naver.com/mnews/article/422/0000607237?sid=101', 'https://n.news.naver.com/mnews/article/214/0001283877?sid=101']
중복 제거 후 행 개수:  3
                  date                                title   
0  2023-07-05 20:18:01  “성과급 반의 반토막 났어요”…삼성전자도 못 피해간 반도체 한파  \
1  2023-07-05 20:16:04        100일간 고강도 원산지 점검…정부, 불안종식 안간힘   
2  2023-07-05 20:11:54   [제보는 MBC] 어느 우수 중소기업의 '지옥같은 회의 시간'   

                                                link   
0  https://n.news.naver.com/mnews/article/009/000...  \
1  https://n.news.naver.com/mnews/article/422/000...   
2  https://n.news.naver.com/mnews/article/214/000...   

                                             content  
0  [\n작년 상반기 100%였지만하반기 50% 이번엔 25%모바일사업부는 50% 결정...  
1  [\n\t\t\t[앵커] 정부는 후쿠시마 오염수 사태와 관련해 불안감 잡기에 안간힘...  
2  [\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n[뉴스데스크]◀ 앵커 ▶우수...  
['https://n.news.naver.com/mnews/article/009/0005154459?sid=101', 'https://n.news.naver.c

In [52]:
news_df.head()

,date,title,link,content
0,2023-07-05 20:18:01,“성과급 반의 반토막 났어요”…삼성전자도 못 피해간 반도체 한파,https://n.news.naver.com/mnews/article/009/000...,[\n작년 상반기 100%였지만하반기 50% 이번엔 25%모바일사업부는 50% 결정...
1,2023-07-05 20:16:04,"100일간 고강도 원산지 점검…정부, 불안종식 안간힘",https://n.news.naver.com/mnews/article/422/000...,[\n\t\t\t[앵커] 정부는 후쿠시마 오염수 사태와 관련해 불안감 잡기에 안간힘...
2,2023-07-05 20:11:54,[제보는 MBC] 어느 우수 중소기업의 '지옥같은 회의 시간',https://n.news.naver.com/mnews/article/214/000...,[\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n[뉴스데스크]◀ 앵커 ▶우수...


In [53]:
###데이터 프레임으로 만들기###
from datetime import date, timedelta
import pandas as pd

#데이터 프레임 만들기
a = pd.DataFrame({'date':news_dates,'title':news_titles,'link':news_urls,'content':news_contents})

pd.concat([news_df, a])

#데이터 프레임 저장
now = datetime.now() 
a.to_csv('C:\\big16\\trading_project\\ssuyan\\REAL_NEWS\\_{}.csv'.format(now.strftime('%Y%m%d_%H%M%S')),encoding='utf-8-sig',index=False)
